In [1]:
from __future__ import division
from authorcode.probability import normal_cdf, inverse_normal_cdf
import math, random

under various assumptions, think of statistics as observations of random variables from known distributions, which allows us to make statements about how likely those are to hold

H_0: coin is fair

H_a: coin is not fair

can approximate bernoulli trials with normal distribution

In [2]:
def normal_approximation_to_binomial(n, p):
    '''finds Normal mu and sigma correseponding to Binomial(n,p)'''
    mu = p * n
    sigma = math.sqrt(p * (1 - p) * n)
    return mu, sigma

when a RV follows a normal distribution, can use normal_cdf to get probability that realized value lies within/outside interval

In [3]:
import inspect

print inspect.getsource(normal_cdf)
print inspect.getsource(inverse_normal_cdf)

def normal_cdf(x, mu=0,sigma=1):
    return (1 + math.erf((x - mu) / math.sqrt(2) / sigma)) / 2  

def inverse_normal_cdf(p, mu=0, sigma=1, tolerance=0.00001):
    """find approximate inverse using binary search"""

    # if not standard, compute standard and rescale
    if mu != 0 or sigma != 1:
        return mu + sigma * inverse_normal_cdf(p, tolerance=tolerance)
    
    low_z, low_p = -10.0, 0            # normal_cdf(-10) is (very close to) 0
    hi_z,  hi_p  =  10.0, 1            # normal_cdf(10)  is (very close to) 1
    while hi_z - low_z > tolerance:
        mid_z = (low_z + hi_z) / 2     # consider the midpoint
        mid_p = normal_cdf(mid_z)      # and the cdf's value there
        if mid_p < p:
            # midpoint is still too low, search above it
            low_z, low_p = mid_z, mid_p
        elif mid_p > p:
            # midpoint is still too high, search below it
            hi_z, hi_p = mid_z, mid_p
        else:
            break

    return mid_z



In [4]:
normal_probability_below = normal_cdf

def normal_probability_above(lo, mu = 0, sigma = 1):
    return 1 - normal_cdf(lo, mu, sigma)

def normal_probability_between(lo, hi, mu = 0, sigma = 1):
    return normal_cdf(hi, mu, sigma) - normal_cdf(lo, mu, sigma)

def normal_probability_outside(lo, hi, mu = 0, sigma = 1):
    return 1 - normal_probability_between(lo, hi, mu, sigma)

In [5]:
def normal_upper_bound(probability, mu = 0, sigma = 1):
    '''returns the z for which P(Z <= z) = probability'''
    return inverse_normal_cdf(probability, mu, sigma)

def normal_lower_bound(probability, mu = 0, sigma = 1):
    '''returns the z for which P(Z >= z) = probability'''
    return inverse_normal_cdf(1 - probability, mu, sigma)

def normal_two_sided_bounds(probability, mu=0, sigma=1):
    """returns the symmetric (about the mean) bounds 
    that contain the specified probability"""
    tail_probability = (1 - probability) / 2

    # upper bound should have tail_probability above it
    upper_bound = normal_lower_bound(tail_probability, mu, sigma)

    # lower bound should have tail_probability below it
    lower_bound = normal_upper_bound(tail_probability, mu, sigma)

    return lower_bound, upper_bound

approximation of 1000 coin flips

In [6]:
mu_0, sigma_0 = normal_approximation_to_binomial(1000, 0.5)
print mu_0
print sigma_0

500.0
15.8113883008


we need to make a decision about SIGNIFICANCE

Q: How willing are we to make a Type I error ("false positive") where we reject H_0 when it's true?

usually 1% or 5% (significance)

In [7]:
normal_two_sided_bounds(0.95, mu_0, sigma_0)

(469.01026640487555, 530.9897335951244)

#### power: probability of NOT making a Type II error
where we fail to reject H_0 when we know it's false

knowing merely that H_0 is not true does not give lots of information about the distribution

let's check what happens if p is really 0.55 and calculate power

95% bounds based on assumption p is 0.5

In [8]:
lo, hi = normal_two_sided_bounds(0.95, mu_0, sigma_0)

actual mu and sigma based on p = 0.55

In [9]:
mu_1, sigma_1 = normal_approximation_to_binomial(1000, 0.55)

a type 2 error means we fail to reject null hypothesis which will happen when X is still in our original interval

In [10]:
# prob of accepting H_0 under actual distribution
type_2_probability = normal_probability_between(lo, hi, mu_1, sigma_1) 
power = 1 - type_2_probability
print 'power', power

power 0.886548001295


H_0: coin not biased towards heads (Prop of heads = 0.5)

H_a: coin biased towards heads (Prop of heads > 0.5)

one-sided test: reject when X much larger than 50, but not when smaller than 50

In [11]:
hi = normal_upper_bound(0.95, mu_0, sigma_0)
print 'hi', hi

type_2_probability = normal_probability_below(hi, mu_1, sigma_1)
power = 1 - type_2_probability
print 'power', power

hi 526.007358524
power 0.936379480331


# p-values
the probability, assuming H_0 is true, that we would see a value at least as extreme as one we observed

In [12]:
def two_sided_p_value(x, mu = 0, sigma = 1):
    if x >= mu:
        # if x is greater than the mean, the tail is what's greater than x
        return 2 * normal_probability_above(x, mu, sigma)
    else:
        # if x is less than the mean, the tail is what's less than x
        return 2 * normal_probability_below(x, mu, sigma)